In [ ]:
#import necessary modules
import numpy as np
import rasterio as rs
import matplotlib.pyplot as plt
import earthpy.spatial as es
import earthpy.plot as ep
from numpy import loadtxt
from osgeo import gdal, gdalconst
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.ensemble import IsolationForest
from sklearn import svm

In [ ]:
#Set imagery as variables
asi = #'Path_to_SingleDateImage_Amsterdam.tif'
ami = #'Path_to_MedianImage_Amsterdam.tif'
msi = #'Path_to_SingleDateImage_Milano.tif'
mmi = #'Path_to_MedianImage_Milano.tif'
bsi = #'Path_to_SingleDateImage_Budapest.tif'
bmi = #'Path_to_MedianImage_Budapest.tif'
filist = [asi, ami, msi, mmi, bsi, bmi]

In [ ]:
def read_data(inras):
    # Read data
    img = gdal.Open(inras, gdal.GA_ReadOnly) 
    bands = [img.GetRasterBand(i).ReadAsArray() for i in range(1, img.RasterCount + 1)]
    img = np.array(bands)
    img = img[0:12,:,:]
    img = np.transpose(img, [1, 2, 0])
    img_rgbnir = img[:,:,[3,2,1,7]]
    return img_rgbnir

#Import Feature extraction
NDWI = loadtxt(#'Path_to_NDWI.csv', delimiter=',')
NDVI = loadtxt(#'Path_to_NDVI.csv', delimiter=',')
EMP = np.load(#'Path_to_EMP.npy')

GLCM_load = loadtxt(#'Path_to_GLCM.csv', delimiter=',')
img = read_data(#imgvariable)
nr, nc, nb = img.shape
nf = 16
GLCM = GLCM_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCM)
GLCM[where_are_NaNs] = 0

In [ ]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(asi)
        stack = np.concatenate((img, EMPas, GLCMas), axis=2)
        stacked = np.dstack((stack, NDVIas, NDWIas))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twita
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30ma
        true2 = Val2GAIAa
        true3 = Val3GHS50a
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score

In [ ]:
#Amsterdam
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('OA' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('P' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('R' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('F' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('AUC' + str(max_key4) + str(max_value4))